When came with a new model. We want to ask some questions. Like what has changed from previous version of model to new version. Is there any preprocessing needed? What are extra libraries that we need to run a new model

And what if when running this new model in production we face some issues and roll back to old model. We need to know where the old model is stored

When doing an ML task, we use the MLFlow Tracking Server to log the parameters, metrics, artifactions and also many different model versions

Once we believe those models are fit for production, then we will "register model" to the MLFlow registry

MLFlow registry is the place where we store the production ready models. So whenver a deployment engineer wants to update the models, they can take a look at the Model Registry to find the new prod ready models

The MLflow Model Registry component is a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLflow Model. It provides model lineage (which MLflow experiment and run produced the model), model versioning, model aliasing, model tagging, and annotations.

Model Registry does not deploy the models, instead it stores the models that are prod ready